<h2 style="color:blue" align="center">Deep Neural Network (DNN) on MNIST</h2>

#### Import the necessary libraries

In [103]:
import tensorflow as tf 
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import time, math 

from tensorflow import keras 
from keras.models import Sequential
from keras.layers import Dense 
from keras.callbacks import Callback, CSVLogger 

#### Load and split the dataset into training and testing set

MNIST - Handwritten digits recognition

In [104]:
(X_train,y_train),(X_test,y_test) = keras.datasets.mnist.load_data()

MNIST - Fashion dataset

In [105]:
#(X_train,y_train),(X_test,y_test) = keras.datasets.fashion_mnist.load_data()

#### Flattening

In [106]:
X_train_flattened = X_train.reshape(len(X_train), 28 * 28)
X_test_flattened = X_test.reshape(len(X_test),  28 * 28) 

#### Normalize the input dataset

In [107]:
X_train_normalized = X_train_flattened / 255
X_test_normalized = X_test_flattened / 255

#### Define DNN model with two layers, optimizer, metrics, and loss function

In [108]:
import random as python_random
python_random.seed(3)
np.random.seed(7)
tf.random.set_seed(13)

def get_model(): 
    model = Sequential([
        # input layer 784 neurons to first hidden layer with 64 neurons
        Dense(64, input_shape = (784,), activation='relu'), 
        # first hidden layer to second hidden layer
        Dense(64, activation='relu'),  
        # Output layer with 10 neurons
        Dense(10, activation='softmax')
    ])
    
    # General SGD
    #opti = keras.optimizers.SGD(learning_rate=0.01)
    
    # SGD with momentum
    #opti = keras.optimizers.SGD(learning_rate=0.01, momentum=0.6)
    
    # SGD with Nesterov momentum 
    #opti = keras.optimizers.SGD(learning_rate=0.01, momentum=0.6, nesterov=True)
    
    # RMSprop 
    #opti = keras.optimizers.RMSprop(learning_rate=0.001, momentum=0.6)
    
    # Adam
    opti = keras.optimizers.Adam(learning_rate=0.001) 
    
    # Adamax
    #opti = keras.optimizers.Adamax(learning_rate=0.001) 
    
    model.compile(
        optimizer = opti,
        loss = 'sparse_categorical_crossentropy',
        metrics = ['accuracy']
    )
    
    return model

#### Custom callbacks

For generic optimizer model

In [109]:
class CustomCallbackGeneric(Callback):  
    # Training stop criteria
    stop_at = 0.99
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')> self.stop_at):  
            self.model.stop_training = True

For HM based optimizer model

In [110]:
class CustomCallbackHM(Callback):  
    # Stop the algorithm when the following accuracy reached 
    stop_at = 0.995  
    
    initial_weights = 0
    previous_weights = 0
    call_hm = 0 
     
    def on_train_begin(self, logs=None):
        self.initial_weights = model_hm.get_weights() 
        self.initial_weights = np.array(self.initial_weights,dtype=object)
        self.previous_weights = self.initial_weights
        # Harmonic mean based weights calculation
        self.call_hm = np.vectorize(self.apply_hm)  

    def on_epoch_end(self, epoch, logs={}):
        # Training stop criteria
        if(logs.get('accuracy')> self.stop_at):  
            self.model.stop_training = True
        
        num_layers = len(model_hm.layers)  
        current_weights = model_hm.get_weights()
        current_weights = np.array(current_weights,dtype=object)        

        for i in range(num_layers):  
            # Harmonic mean based weights calculation
            tensor1 = tf.convert_to_tensor(self.previous_weights[i*2])
            tensor2 = tf.convert_to_tensor(current_weights[i*2])
            current_weights[i*2] = self.call_hm(tensor1, tensor2)   
            
        # Updating the model with new weights
        model_hm.set_weights(current_weights.tolist())
        self.previous_weights = current_weights
        
    def apply_hm(self, v1, v2):     
        if v1==0 or v2==0:
            return v2
        elif v1>0 and v2>0:
            hm = 2*v1*v2/(v1+v2)
            min1 = min(v1,v2)
            diff = abs(hm-min1)
            if v2 > v1:
                return v2 + diff
            else:
                return v2 - diff
        elif v1<0 and v2<0:
            hm = 2*v1*v2/(v1+v2)
            max1 = max(v1,v2)
            diff = abs(hm-max1)
            if v2 > v1:
                return v2 + diff
            else:
                return v2 - diff
        else:
            return v2  

To record loss and accuracy in CSV file

In [111]:
logger_generic_model = CSVLogger('Generic_model_MNIST.csv', append=False, separator=',')
logger_hm_model = CSVLogger('HM_model_MNIST.csv', append=False, separator=',')

#### Training

Get a model to assign same weights to model with and without HM

In [112]:
model = get_model() 
weights = model.get_weights() 
model_wihtout_hm = get_model()
model_wihtout_hm.set_weights(weights) 
model_hm = get_model()
model_hm.set_weights(weights) 

Generic optimizer model

In [116]:
st = time.time()
model_wihtout_hm.fit(X_train_normalized, y_train, epochs = 200, verbose=1, callbacks=[CustomCallbackGeneric(), logger_generic_model], batch_size = X_train.shape[0]) 
et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

Epoch 1/200
1/1 [==============================] - 1s 693ms/step - loss: 2.3563 - accuracy: 0.1617
Epoch 2/200
1/1 [==============================] - 0s 173ms/step - loss: 2.2024 - accuracy: 0.2233
Epoch 3/200
1/1 [==============================] - 0s 175ms/step - loss: 2.0940 - accuracy: 0.2942
Epoch 4/200
1/1 [==============================] - 0s 185ms/step - loss: 1.9998 - accuracy: 0.3410
Epoch 5/200
1/1 [==============================] - 0s 174ms/step - loss: 1.9052 - accuracy: 0.3736
Epoch 6/200
1/1 [==============================] - 0s 178ms/step - loss: 1.8137 - accuracy: 0.4196
Epoch 7/200
1/1 [==============================] - 0s 189ms/step - loss: 1.7220 - accuracy: 0.4589
Epoch 8/200
1/1 [==============================] - 0s 200ms/step - loss: 1.6288 - accuracy: 0.4893
Epoch 9/200
1/1 [==============================] - 0s 189ms/step - loss: 1.5386 - accuracy: 0.5350
Epoch 10/200
1/1 [==============================] - 0s 193ms/step - loss: 1.4545 - accuracy: 0.5891
Epoch 11/

1/1 [==============================] - 0s 191ms/step - loss: 0.4708 - accuracy: 0.8397
Epoch 84/200
1/1 [==============================] - 0s 211ms/step - loss: 0.4687 - accuracy: 0.8403
Epoch 85/200
1/1 [==============================] - 0s 178ms/step - loss: 0.4666 - accuracy: 0.8411
Epoch 86/200
1/1 [==============================] - 0s 179ms/step - loss: 0.4645 - accuracy: 0.8417
Epoch 87/200
1/1 [==============================] - 0s 228ms/step - loss: 0.4625 - accuracy: 0.8422
Epoch 88/200
1/1 [==============================] - 0s 192ms/step - loss: 0.4606 - accuracy: 0.8428
Epoch 89/200
1/1 [==============================] - 0s 203ms/step - loss: 0.4587 - accuracy: 0.8434
Epoch 90/200
1/1 [==============================] - 0s 189ms/step - loss: 0.4568 - accuracy: 0.8441
Epoch 91/200
1/1 [==============================] - 0s 195ms/step - loss: 0.4549 - accuracy: 0.8444
Epoch 92/200
1/1 [==============================] - 0s 202ms/step - loss: 0.4531 - accuracy: 0.8448
Epoch 93/200


HM based optimizer model

In [117]:
st = time.time()
model_hm.fit(X_train_normalized, y_train, epochs = 175, verbose=1, callbacks=[CustomCallbackHM(),logger_hm_model], batch_size = X_train.shape[0]) 
et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

Epoch 1/175
1/1 [==============================] - 1s 787ms/step - loss: 2.3563 - accuracy: 0.1617
Epoch 2/175
1/1 [==============================] - 0s 239ms/step - loss: 2.1510 - accuracy: 0.2585
Epoch 3/175
1/1 [==============================] - 0s 256ms/step - loss: 2.0183 - accuracy: 0.3388
Epoch 4/175
1/1 [==============================] - 0s 240ms/step - loss: 1.8870 - accuracy: 0.3788
Epoch 5/175
1/1 [==============================] - 0s 246ms/step - loss: 1.7588 - accuracy: 0.4441
Epoch 6/175
1/1 [==============================] - 0s 236ms/step - loss: 1.6277 - accuracy: 0.4893
Epoch 7/175
1/1 [==============================] - 0s 233ms/step - loss: 1.5041 - accuracy: 0.5491
Epoch 8/175
1/1 [==============================] - 0s 263ms/step - loss: 1.3952 - accuracy: 0.6077
Epoch 9/175
1/1 [==============================] - 0s 238ms/step - loss: 1.2987 - accuracy: 0.6316
Epoch 10/175
1/1 [==============================] - 0s 264ms/step - loss: 1.2150 - accuracy: 0.6418
Epoch 11/

1/1 [==============================] - 0s 206ms/step - loss: 0.4310 - accuracy: 0.8524
Epoch 84/175
1/1 [==============================] - 0s 212ms/step - loss: 0.4298 - accuracy: 0.8530
Epoch 85/175
1/1 [==============================] - 0s 222ms/step - loss: 0.4276 - accuracy: 0.8534
Epoch 86/175
1/1 [==============================] - 0s 210ms/step - loss: 0.4257 - accuracy: 0.8540
Epoch 87/175
1/1 [==============================] - 0s 200ms/step - loss: 0.4246 - accuracy: 0.8546
Epoch 88/175
1/1 [==============================] - 0s 206ms/step - loss: 0.4231 - accuracy: 0.8548
Epoch 89/175
1/1 [==============================] - 0s 214ms/step - loss: 0.4213 - accuracy: 0.8551
Epoch 90/175
1/1 [==============================] - 0s 217ms/step - loss: 0.4200 - accuracy: 0.8558
Epoch 91/175
1/1 [==============================] - 0s 212ms/step - loss: 0.4188 - accuracy: 0.8557
Epoch 92/175
1/1 [==============================] - 0s 268ms/step - loss: 0.4172 - accuracy: 0.8568
Epoch 93/175


Model summary

In [118]:
model_hm.summary()

Model: "sequential_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_90 (Dense)            (None, 64)                50240     
                                                                 
 dense_91 (Dense)            (None, 64)                4160      
                                                                 
 dense_92 (Dense)            (None, 10)                650       
                                                                 
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


###### Testing the model

Generic opimizer model

In [119]:
model_wihtout_hm.evaluate(X_test_normalized, y_test)

313/313 [==============================] - 0s 1ms/step - loss: 0.3996 - accuracy: 0.8583


[0.3995527923107147, 0.858299970626831]

HM based optimizer model

In [120]:
model_hm.evaluate(X_test_normalized, y_test)

313/313 [==============================] - 0s 1ms/step - loss: 0.3986 - accuracy: 0.8616


[0.3985791504383087, 0.8615999817848206]